In [1]:
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import glob
import os

# make function

In [2]:
def origin_rate(df):
    df_xy = df[[(a,b) for a,b in df.columns if b in ('x', 'y')]]
    df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
    df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
    return df_xy

In [3]:
def box_perspective(df_xy):
    for idx in range(len(df_xy)):
        x_min = df_xy.loc[idx, (slice(None), 'x')].min()
        x_max = df_xy.loc[idx, (slice(None), 'x')].max()
        y_min = df_xy.loc[idx, (slice(None), 'y')].min()
        y_max = df_xy.loc[idx, (slice(None), 'y')].max()
        
        if x_max != 0 and y_max != 0:
            x_rate = 640/((x_max+20)-(x_min-20))
            y_rate = 720/((y_max+20)-(y_min-20))

            df_xy.loc[idx, (slice(None), 'x')] = df_xy.loc[idx, (slice(None), 'x')].apply(lambda x : ((x-(x_min-20))*x_rate)/640)
            df_xy.loc[idx, (slice(None), 'y')] = df_xy.loc[idx, (slice(None), 'y')].apply(lambda x : ((x-(y_min-20))*y_rate)/720)
        
    return df_xy

In [4]:
def vector_transform(joint):    
    # Compute angles between joints
    v1 = joint[[13, 23, 14, 24, 15, 16, 17, 19, 21, 18, 20, 22, 25, 26, 27, 28, 29, 31, 30, 32], :] # Parent joint
    v2 = joint[[11, 11, 12, 12, 13, 14, 15, 15, 15, 16, 16, 16, 23, 24, 25, 26, 27, 27, 28, 28], :] # Child joint
    v = v2 - v1 # [20, 2]  
    # Normalize v
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
    
    return v

# make vectors from skeleton csv

In [6]:
paths = glob.glob("./video/*.csv")

In [7]:
paths

['./video\\baby_skeleton.csv',
 './video\\bottom_skeleton.csv',
 './video\\goodmorning_skeleton.csv',
 './video\\growup_skeleton.csv',
 './video\\jungle_dance_skeleton.csv',
 './video\\monster_skeleton.csv',
 './video\\octopus_skeleton.csv',
 './video\\pinkpong_skeleton.csv',
 './video\\poo_skeleton.csv',
 './video\\volcano_skeleton.csv']

In [8]:
for path in paths:
    name = path.split("\\")[1].split("_")[0]
    df = pd.read_csv(path, header=[0,1])
    df_xy = origin_rate(df)
    df_bp = box_perspective(df_xy)
    vector_list = []
    for idx in range(df_bp.shape[0]):
        joint = df_bp.iloc[idx].values.reshape(33,2)
        vector = vector_transform(joint)
        vector_list.append(vector)
    vector_np = np.array(vector_list)
    np.save(os.path.join('video', f'vector_{name}'), vector_np)

C:\Users\조\AppData\Local\Temp\ipykernel_17512\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\조\AppData\Local\Temp\ipykernel_17512\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\조\AppData\Local\Temp\ipykernel_17512\1296948987.py:7: RuntimeWarning: invalid value encountered in tr

C:\Users\조\AppData\Local\Temp\ipykernel_17512\1296948987.py:7: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
C:\Users\조\AppData\Local\Temp\ipykernel_17512\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\조\AppData\Local\Temp\ipykernel_17512\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')